In [61]:
#Packages Needed
import pandas as pd
import numpy as np
import re

### This analysis is based on the All-Star data from 2000 - 2016

The .csv file was created by 

In [9]:
df = pd.read_csv('gmoney-nba-all-stars-2000-2016/data/nba_all_star_games.csv')
df.tail()

,year,player,pos,ht,wt,team,selection_type,nba_draft_status,nationality
434,2000,Antonio McDyess,F,6-9,245,Denver Nuggets,Western All-Star Coaches Selection,1995 Rnd 1 Pick 2,United States
435,2000,Gary Payton,G,6-4,180,Seattle SuperSonics,Western All-Star Coaches Selection,1990 Rnd 1 Pick 2,United States
436,2000,Rasheed Wallace,FC,6-11,230,Portland Trail Blazers,Western All-Star Coaches Selection,1995 Rnd 1 Pick 4,United States
437,2000,David Robinson,C,7-1,250,San Antonio Spurs,Western All-Star Coaches Selection,1987 Rnd 1 Pick 1,United States
438,2000,Vlade Divac,C,7-1,260,Sacramento Kings,Western All-Star Replacement Selection,1989 Rnd 1 Pick 26,Serbia


In [10]:
#Needed so repeated player data doesn't skew the data
filtered_df = df.drop_duplicates(subset = 'player')
filtered_df = filtered_df.reset_index()
filtered_df.tail()

,index,year,player,pos,ht,wt,team,selection_type,nba_draft_status,nationality
118,425,2000,Allan Houston,G,6-6,205,New York Knicks,Eastern All-Star Coaches Selection,1993 Rnd 1 Pick 11,United States
119,431,2000,Michael Finley,G-F,6-7,225,Dallas Mavericks,Western All-Star Coaches Selection,1995 Rnd 1 Pick 21,United States
120,434,2000,Antonio McDyess,F,6-9,245,Denver Nuggets,Western All-Star Coaches Selection,1995 Rnd 1 Pick 2,United States
121,437,2000,David Robinson,C,7-1,250,San Antonio Spurs,Western All-Star Coaches Selection,1987 Rnd 1 Pick 1,United States
122,438,2000,Vlade Divac,C,7-1,260,Sacramento Kings,Western All-Star Replacement Selection,1989 Rnd 1 Pick 26,Serbia


#### This cell just checks the nations that players have come from

In [11]:
filtered_df["Multi National"] = filtered_df.apply(lambda row: 1 if '\n' in row['nationality'] else 0, axis = 1)
num_of_multi_nationals = filtered_df["Multi National"].sum()

countries_w_AS = filtered_df.drop_duplicates(subset = 'nationality')['nationality'].tolist()
multi_nationals = []
need_to_rm = []
for country in countries_w_AS:
    if '\n' in country:
        temp = country.split('\n')
        multi_nationals += temp
        need_to_rm.append(country)
#         print(repr(country))
for country in need_to_rm:
    countries_w_AS.remove(country)

# print(multi_nationals)
countries_w_AS = list(set(countries_w_AS+multi_nationals))

print("There have been", num_of_multi_nationals,"all-stars that represent multiple nations from 2000-2016\n")
print("Countries that have ties with all starts include:")
build_str = ''
for i in range(0, len(countries_w_AS)-1):
#     print(countries_w_AS[i])
    build_str += countries_w_AS[i] + ', '
build_str += 'and ' + countries_w_AS[-1] + '.'
print(build_str)

There have been 10 all-stars that represent multiple nations from 2000-2016

Countries that have ties with all starts include:
Lithuania, Canada, United States, Australia, Democratic Republic of the Congo, Argentina, Russia, China, South Sudan, France, Nigeria, South Africa, Spain, Greece, Germany, England, Turkey, Dominican Republic, and Serbia.


In [146]:
#Takes input as an int for inches
#Returns a string in the format of 'Feet-Inches'
def inchesToFeet(inches):
    ft = int(inches)//12
    i = inches%12
    return str(ft)+'-'+str(i)
#Takes input as a string in the format of 'Feet-Inches'
#Returns an int for inches
def feetToInches(ht):
    digits = re.findall(r'[0-9]+',ht)
    feet = int(digits[0])
    inches = int(digits[1])
    return (feet*12) + inches

In [24]:
'''
True position consists of 3 different classifications:
    G -> Guard  Traditional PG, and Combo Guards
    W -> Wing   Traditional SF, Traditional SG, SF/PF tweeners
    B -> Big    Traditional PF, Traditional C, Stretch 4's
'''
filtered_df["true pos"] = filtered_df["pos"]
filtered_df["true pos"]  = filtered_df.apply(lambda row: 'G' if 'G' in row['pos'] else row['true pos'], axis = 1)
filtered_df["true pos"]  = filtered_df.apply(lambda row: 'W' if 'F' in row['pos'] else row['true pos'], axis = 1)
filtered_df["true pos"]  = filtered_df.apply(lambda row: 'B' if 'PF' in row['pos'] else row['true pos'], axis = 1)
filtered_df["true pos"]  = filtered_df.apply(lambda row: 'B' if 'C' in row['pos'] else row['true pos'], axis = 1)
# filtered_df

In [149]:
heightInInches = pd.DataFrame()
heightInInches["true pos"] = filtered_df["true pos"]
heightInInches["ht (inches)"] = filtered_df.apply(lambda row: feetToInches(row['ht']), axis = 1)
heightInInches["guard hts"] = heightInInches.apply(lambda row: row['ht (inches)'] if row['true pos'] == 'G' else 0, axis = 1).replace(0, np.nan)
heightInInches["wing hts"] = heightInInches.apply(lambda row: row['ht (inches)'] if row['true pos'] == 'W' else 0, axis = 1).replace(0, np.nan)
heightInInches["big hts"] = heightInInches.apply(lambda row: row['ht (inches)'] if row['true pos'] == 'B' else 0, axis = 1).replace(0, np.nan)

avgHeightAll = heightInInches["ht (inches)"].mean()
avgHeightG = heightInInches["guard hts"].mean()
avgHeightW = heightInInches['wing hts'].mean()
avgHeightB = heightInInches['big hts'].mean()
# heightInInches.mean()
print("The average height of a Guard was: "+ inchesToFeet(avgHeightG))
print("The average height of a Wing was: "+ inchesToFeet(avgHeightW))
print("The average height of a Big was: "+ inchesToFeet(avgHeightB))

The average height of a Guard was: 6-3.3414634146341484
The average height of a Wing was: 6-7.885714285714286
The average height of a Big was: 6-11.0


In [159]:
def draftPosConverter(pos):
    if(type(pos) == int or type(pos) == float):
#         print("Pos is an int")
        return "Rnd " + str(int(((pos-1)//30)+1)) + " Pick " + str((pos-1)%30+1)
    elif(type(pos) == str):
#         print("Pos is a str")
        rnd = re.findall(r'Rnd\s\d', pos)
        if len(rnd) == 0:
            return np.nan
        rnd = int(rnd[0][3:])
        pick = int(re.findall(r'Pick\s\d+$', pos)[0][4:])
        return (rnd - 1)*30 + pick
    else:
        return np.nan

In [160]:
draftPos = pd.DataFrame()
draftPos['undrafted'] = filtered_df.apply(lambda row: 1 if 'Undrafted' in row['nba_draft_status'] else 0, axis = 1)
num_UDFA = draftPos['undrafted'].cumsum().tolist()[-1]
print('%d All-Stars from 2000-2016 were undrafted free agents' % (num_UDFA))
draftPos['draft_pos'] = filtered_df.apply(lambda row: draftPosConverter(row['nba_draft_status']), axis = 1)
print("The average draft position of All-Stars from 2000-2016 discounting UDFA was: " + \
      draftPosConverter(draftPos['draft_pos'].mean()))

2 All-Stars from 2000-2016 were undrafted free agents
The average draft position of All-Stars from 2000-2016 discounting UDFA was: Rnd 1 Pick 13.561983471074381
